In [ ]:
%profile dev
%idle_timeout 60
%iam_role arn:aws:iam::866525145675:role/grsi-dp-extract-glue-role-development
%number_of_workers 5
%worker_type G.1X
%security_config grsi-dp-security-config-development
%region eu-west-1
%glue_version 3.0
%connections GeniusAnalyticalRep-RDS-test
%tags {"lm_troux_uid": "B3439EB1-509F-4193-B1B3-2023D75CBD29", "organization_guid" : "2326c977-2f8d-4d97-8f13-a7ca2fd5d5db"}

In [ ]:
import sys
import boto3

from awsglue.context import GlueContext
from awsglue.dynamicframe import DynamicFrame
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext

In [ ]:
glueContext = GlueContext(SparkContext.getOrCreate())
spark = glueContext.spark_session
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
spark.sql("set spark.sql.legacy.parquet.int96RebaseModeInWrite=CORRECTED")

In [ ]:
sys.argv.append("--JOB_NAME")
sys.argv.append("claim-center-extract")
sys.argv.append("--ACCOUNT_ID")
sys.argv.append("866525145675")
sys.argv.append("--AWS_REGION")
sys.argv.append("eu-west-1")
sys.argv.append("--BILZ_CONNECTION")
sys.argv.append("GeniusAnalyticalRep-RDS-test")
sys.argv.append("--APP_ENV")
sys.argv.append("development")


In [ ]:
args = getResolvedOptions(
    sys.argv,
    [
        "JOB_NAME",
        "AWS_REGION",
        "ACCOUNT_ID",
        "APP_ENV",
        "BILZ_CONNECTION"
    ],
)

environment = args["APP_ENV"]
bilz_connection = args["BILZ_CONNECTION"]
region = args["AWS_REGION"]
acc_id = args["ACCOUNT_ID"]

s3 = boto3.resource("s3")
bucket_name = f"grsi-dp-source-data-{environment}"
bucket = s3.Bucket(bucket_name)


In [ ]:
cc_tables = [
    "cc_claim",
    "cc_policy",
    "cc_group",
    "cc_address",
    "cc_matter",
    "cctl_country",
    "cctl_state",
    "cc_catastrophe",
    "cc_ext_claimevent",
    "cc_user",
    "cc_contact",
    "cc_history",
    "cctl_extcollcode",
    "cctl_extcollcode2",
    "cctl_extcollcode3",
    "cctl_ext_collcode3_pi",
    "cctl_extcollcode4",
    "cctl_ext_projecttype",
    "cctl_ext_typeofwork2",
    "cctl_ext_policycovsectionusa",
    "cctl_extcollcode5",
    "cctl_ecfcauseoflosscode_ext",
    "cctl_claimstate",
    "cctl_claimtier",
    "cctl_extclaimtype",
    "cctl_ext_handledby",
    "cctl_ext_isseveritycode",
    "cctl_extseverity",
    "cctl_claimsecuritytype",
    "cctl_underwritingcompanytype",
    "cctl_ext_product",
    "cctl_lmclasscode_syn",
    "cctl_ext_losscodeset",
    "cctl_extcyberconsideration",
    "cctl_extransomwareaspect",
    "cctl_ext_claimsorganization",
    "cctl_ext_coverageindicatorpr",
    "cctl_extcoverageindicator",
    "cctl_ext_deductibletype",
    "cctl_faultrating",
    "cctl_ext_handlingtype",
    "cctl_largelossnotificationstat",
    "cctl_ext_isolosslocation",
    "cctl_ext_cargo",
    "cctl_ext_lossevent",
    "cctl_personrelationtype",
    "cctl_howreportedtype",
    "cctl_ext_proceedingtype",
    "cctl_ext_proceedingspr",
    "cc_subrogationsummary",
    "cc_subrogation",
    "cctl_subrogationstatus",
    "cc_exposure",
    "cctl_ext_policyorigin",
    "cctl_historytype",
    "cctl_customhistorytype",
    "cc_claim_log",
    "cc_parentgroup"
]



In [ ]:
genius_details = {
    "source_db":"bi_lz",
    "target_db": f"grsi_dp_{environment}",
    "source_schema":"genius",
    "target_schema":"staging",
    "table_prefix":"genius",
    "table_names" : [
        "gyllsmbf",
        ]
}
cc_details = {
    "source_db":"bi_lz",
    "target_db": f"grsi_dp_{environment}",
    "source_schema": "claimcenter",
    "target_schema":"staging",
    "table_prefix":"claim_center",
    "table_names" : cc_tables
}

bi_lz_schemas = [
#     genius_details,
    cc_details
]

In [ ]:
for schema in bi_lz_schemas:
    for table in schema['table_names']:
        source_table = f"{schema['source_db']}_{schema['source_schema']}_{table}"
        s3_location = f"{schema['source_db']}/{schema['source_schema']}/{table}"
        
        for obj in bucket.objects.filter(Prefix=s3_location):
            s3.Object(bucket_name, obj.key).delete()
            print(obj.key)

        dynamic_frame = glueContext.create_dynamic_frame.from_catalog(
            database=f"grsi-dp-claim-center-{environment}", table_name=source_table
        )

        target_location = f"s3://{bucket_name}/{schema['source_db']}/{schema['source_schema']}/{table}/"

        glueContext.write_dynamic_frame.from_options(
            frame=dynamic_frame.coalesce(1),
            connection_type="s3",
            connection_options={"path": target_location},
            format="parquet",
        )


In [ ]:
%stop_session